In [1]:
import vocab
import re

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.randint(0,0, (1, ))

RuntimeError: random_ expects 'from' to be less than 'to', but got from=0 >= to=0